In [ ]:
from typing import Annotated, TypedDict
from langgraph.graph import StateGraph, add_messages, END
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_tavily import TavilySearch
from langgraph.checkpoint.memory import MemorySaver
from uuid import uuid4
import json

In [ ]:
load_dotenv()

In [ ]:
llm = ChatOpenAI(model = "gpt-4o-mini")
search_tool = TavilySearch(max_results=4)
tools = [search_tool]
memory = MemorySaver()

In [ ]:
llm_with_tools = llm.bind_tools(tools = tools)

In [ ]:
from langchain_core.messages import HumanMessage, ToolMessage
class State(TypedDict):
    messages: Annotated[list, add_messages]

async def model(state: State):
    result = await llm_with_tools.ainvoke(state['messages'])
    return {
        "messages": [result]
    }

async def tools_router(state: State):
    last_message = state['messages'][-1]

    if (hasattr(last_message, "tool_calls") and len(last_message.tool_calls) > 0):
        return "tool_node"
    else:
        return END

    
async def tool_node(state: State):
    """Custom tool node that handles tool calls from LLM"""
    tool_calls = state["messages"][-1].tool_calls

    tool_messages = []

    # process each tool call
    for tool_call in tool_calls:
        tool_name = tool_call["name"]
        tool_args = tool_call["args"]
        tool_id = tool_call["id"]

        if tool_name == "tavily_search":
            search_results = await search_tool.ainvoke(tool_args)
            tool_message = ToolMessage(
                content=str(search_results),
                tool_call_id = tool_id,
                name = tool_name
            )
            tool_messages.append(tool_message)
    
    return {
        "messages": tool_messages
    }

In [ ]:
graph_builder = StateGraph(State)
graph_builder.add_node("model", model)
graph_builder.add_node("tool_node", tool_node)
graph_builder.set_entry_point("model")

graph_builder.add_edge("tool_node", "model")
graph_builder.add_conditional_edges(
    "model",
    tools_router,
    {
        "tool_node": "tool_node",
        END: END
    }
)

graph = graph_builder.compile(checkpointer=memory)

In [ ]:
from IPython.display import Image, display
from langchain_core.runnables.graph import MermaidDrawMethod

display(
    Image(
        graph.get_graph().draw_mermaid_png(
            draw_method=MermaidDrawMethod.API
        )
    )
)

In [ ]:
thread_id = uuid4()

config = {
    "configurable": {
        "thread_id": thread_id
    }
}

response = await graph.ainvoke({
    "messages": [HumanMessage(content="What's current weather in bangkok, thailand?")]
}, config=config)

In [ ]:
thread_id = uuid4()

config = {
    "configurable": {
        "thread_id": thread_id
    }
}

async for event in graph.astream_events({
    "messages": [HumanMessage(content="What's current weather in bangkok, thailand?")]
}, config=config, version="v2"):
    if event['event'] == "on_chat_model_stream":
        print(event['data']['chunk'].content, end = "", flush = True)